In [1]:
import pymongo
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb=myclient['smartshark_2_1']
mycol = mydb["project"]

projects=[]
list2=[]
list3=[]
list4=[]
detailedissues=[]
joinedissues = mydb.project.aggregate([{"$lookup":{
            "from": "mailing_list",       # other table name
            "localField": "_id",        # key field in collection 2
            "foreignField": "project_id",      # key field in collection 1
            "as": "linked_collections"   # alias for resulting table
        }},
        {   "$unwind":"$linked_collections"},
                                
         {"$lookup":{
            "from": "message",       # other table name
            "localField": "linked_collections._id",        # key field in collection 2
            "foreignField": "mailing_list_id",      # key field in collection 1
            "as": "linked_collections2"   # alias for resulting table
        }},
       
         {   "$unwind":"$linked_collections2" },                             
        {"$project": {"_id":1,"name":1,"linked_collections2.body":1}}
        ])


listofmessages=[]
i=6
while i>1:

 for docs in joinedissues:
      list3.append((docs["_id"],docs["name"],docs["linked_collections2"]))
      i=i-1

for project in mycol.find():
 projects.append(project["_id"])


In [3]:
projectmail=[]
for projectid in projects:
    detailedmessage=[]
  

    hit1=0
    for item,name,comment in list3:
        
      
        if projectid==item:
            hit1=hit1+1
            detailedmessage.append(comment)
    projectmail.append(detailedmessage)

    listofmessages.append(hit1)
    

In [ ]:
from nltk.corpus import names
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
import string
from sklearn.feature_extraction import text
# from nltk.corpus import stopwords
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize, pos_tag

# all_names = set(names.words())
lemmatizer = WordNetLemmatizer()

ignore = ['{','}','.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}',"%",'\n','\n2','ok','yes','no','thanks','good','sure','did','Thanks','Sure','welcome']  
stop_words = text.ENGLISH_STOP_WORDS.union(ignore)

cv = CountVectorizer(stop_words=stop_words, max_features=500)

cleaned=[]
def letters_only(astr):
 return astr.isalpha()

exclude = set(string.punctuation)

def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ' or pos[:2]=='VB'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)
def isoneword(text):
    if len(text)>2:
        return True
    else:
        return False

for test in projectmail:
    for post in test:
      values=str(post.get('body'))
      sentence=nouns_adj(values)
      cleaned.append(' '.join([lemmatizer.lemmatize(word) for word in sentence.split() if  letters_only(word) and word not in stop_words and word not in ignore and word not in exclude]))


In [ ]:
from sklearn.decomposition import NMF
nmf = NMF(n_components=100, random_state=42).fit(transformed)
for topic_idx, topic in enumerate(nmf.components_):
 label = '{}: '.format(topic_idx)
 print(label, " ".join([cv.get_feature_names_out()[i]
 for i in topic.argsort()[:-9:-1]]))

In [ ]:
from textblob import TextBlob
test=projectmail[0]
pol = lambda x: TextBlob(x).sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity
subresults=[]
polresults=[]
for test in projectmail[0]
 for post in test:
     values=str(post.get('body'))
     
     polarity=TextBlob(values).sentiment.subjectivity
     subjectivity=TextBlob(values).sentiment.polarity
     subresults.append(subjectivity)
     polresults.append(polarity)

one=np.asarray(subresults)
two=np.asarray(polresults)
print(np.mean(one,axis=None))
print(np.mean(two,axis=None))

In [ ]:
from tqdm import tqdm
import gensim

import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS

import pprint
final=[doc.split() for doc in cleaned]
dictionary = gensim.corpora.Dictionary(final)
dictionary.filter_extremes(no_below=15, no_above=0.4, keep_n=120000)
corpus = [dictionary.doc2bow(doc) for doc in final]


lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=0,
                                           
                                           passes=40,
                                           alpha='auto',
                                           per_word_topics=True)
lda_model.print_topics()
